In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
path = "c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)
from selenium import webdriver
import urllib.request
import urllib.parse
import time
import re #정규식

## 강좌 크롤링 

In [ ]:
count =1
url = 'https://everytime.kr/timetable'
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.get(url)
time.sleep(3)
user_id = input('에브리타임 아이디를 입력해주세요.: ')
et_login = driver.find_element_by_name("userid")
et_login.clear()
et_login.send_keys(user_id)

user_pw = input('에브리타임 비밀번호를 입력해주세요.: ')
et_login = driver.find_element_by_name("password")
et_login.clear()
et_login.send_keys(user_pw) #자신의 비번을 넣으세요

driver.find_element_by_xpath("""//*[@id="container"]/form/p[3]/input""").click()
time.sleep(3)
while True:
    try:
        driver.find_element_by_xpath("""//*[@id="sheet"]/ul/li[3]/a""").click()
        time.sleep(2)
        break
    except:
        continue
timetable_all = pd.DataFrame()
timetable_all
graph = {}
while True:
    try:
        driver.find_element_by_xpath("""//*[@id="container"]/ul/li[1]""").click()
        time.sleep(2)
        break
    except:
        continue
for i in range(1):
    while True:
        try:
            driver.find_element_by_xpath("""//*[@id="subjects"]/div[1]/a[4]""").click()
            time.sleep(2)
            break
        except:
            continue
    while i == 0:
        try:
            driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/div/ul/li[3]""").click()
            time.sleep(2)
            break
        except:
            continue
    while True:
        try:
            driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/div/ul/ul[3]/li["""+str((i+1))+"""]""").click()
            time.sleep(2)
            break
        except:
            continue
#     while True:
#         try:
#             driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/div/ul/ul[2]/ul["""+str((i+1))+"""]/li""").click()
#             time.sleep(2)
#             break
#         except:
#             continue
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    contents = []
    tmp = soup.find('div','list').find('tbody').find_all('td')
 
    for i in tmp:
        
        tmp = i.get_text()
        if (str(i)[14:18]=='star'):
            address = i.find('a')['href']
            url = 'https://everytime.kr' + str(address)
            html = urllib.request.urlopen(url)
            soup = BeautifulSoup(html, 'html.parser')
            #infoArea = soup.select('#container > div.side.article > div.rating > div.details > p:nth-child(1) > span')
            print(str(count) + '\t' + url)
            
            #print(infoArea)
            count = count + 1           
            
        contents.append(tmp)
    contents = np.array(contents)
    
    soup = BeautifulSoup(page, "html.parser")
    column = []
    tmp = soup.find('div','list').find('thead').find_all('th')
    for i in tmp:
        tmp = str(i).lstrip('<th>').split('<div>')
        column.append(tmp[0])
    timetable = pd.DataFrame(contents.reshape(len(contents)//12,12), columns=column)
    
    
    #del(timetable['계획서']) ;del(timetable['강의평'])
    major = soup.find('a','item active').get_text().split(':')[1]
    graph[str(major)] = len(timetable)
    timetable_all = pd.concat([timetable_all,timetable],ignore_index=True)
    timetable.to_excel('./data/20년 2학기 '+major+' 강좌.xlsx',encoding = 'utf8',sheet_name=major,index = False) # 학과 별로 저장
time.sleep(2)
driver.close()




## 해쉬태그용 크롤링 

In [ ]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse

# Chrome의 경우 | 아까 받은 chromedriver의 위치를 지정해준다.
driver = webdriver.Chrome(executable_path=r'./driver/chromedriver')

# url에 접근한다.
driver.get('https://everytime.kr/login')
# 암묵적으로 웹 자원 로드를 위해 5초까지 기다려 준다.
driver.implicitly_wait(5)

# 아이디/비밀번호를 입력해준다.
driver.find_element_by_name('userid').send_keys('자기 아이디 넣기')
driver.find_element_by_name('password').send_keys('자기 패스워드 넣기')

# 로그인 버튼을 눌러주자.
driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()

driver.get('https://everytime.kr/lecture/view/1854171')

page = driver.page_source
soup = BeautifulSoup(page, "html.parser")
infoArea = soup.select('#container > div.side.article > div.rating > div.details > p:nth-child(1) > span')
print(infoArea)


driver.close()


## 파일들 확인  

In [ ]:
tempTt.tail()

In [ ]:
timetable.tail()

In [ ]:
timetable = pd.read_excel('./data/20년 2학기 산업경영공학과 강좌.xlsx') 
timetable.head()

In [ ]:
timetable = pd.read_excel('./data/20년 2학기 공과대학 강좌.xlsx') 
timetable.tail()

## 전체 강좌 크롤링

In [ ]:
#-*- coding:utf-8 -*-
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
from openpyxl import Workbook,load_workbook

# Chrome의 경우 | 아까 받은 chromedriver의 위치를 지정해준다.
driver = webdriver.Chrome(executable_path=r'./driver/chromedriver')

# url에 접근한다.
driver.get('https://everytime.kr/login')
# 암묵적으로 웹 자원 로드를 위해 5초까지 기다려 준다.
driver.implicitly_wait(5)

# 아이디/비밀번호를 입력해준다.
driver.find_element_by_name('userid').send_keys('본인 아이디 입력')
driver.find_element_by_name('password').send_keys('본인 비밀번호 입력 ')

# 로그인 버튼을 눌러주자.
driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()

driver.get('https://everytime.kr/timetable')

#수업 목록에서 검색 클릭
driver.find_element_by_xpath('//*[@id="container"]/ul/li[1]').click()

#팝업창 닫기
sleep(4)
driver.find_element_by_xpath('//*[@id="sheet"]/ul/li[3]/a').click()

pre_count = 0
#스크롤 맨아래로 내리기
while True:
    #tr요소 접근
    element = driver.find_elements_by_css_selector("#subjects > div.list > table > tbody > tr")

    # tr 마지막 요소 접근
    result = element[-1]
    #마지막요소에 focus주기
    driver.execute_script('arguments[0].scrollIntoView(true);',result)
    sleep(2)

    #현재 접근한 요소의 갯수
    current_count = len(element)
    if pre_count == current_count:
        break
    #같지않다면
    pre_count = current_count


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

trs = soup.select('#subjects > div.list > table > tbody > tr')

results = []

for tr in trs:
    result=[]
    tds = tr.select('#subjects > div.list > table > tbody > tr > td')
    result.append(tds[0].text) #조회
    result.append(tds[1].text) #대상학년
    result.append(tds[2].text) #학수번호
    result.append(tds[3].text) #강의이름
    result.append(tds[4].text) #교수님성함
    result.append(tds[5].text) #학점
    result.append(tds[6].text) #수업시간
    result.append(tds[7].text) #강의유형
    result.append(tds[9].text) #담은 인원
    result.append(tds[11].text) # 특이사항
    results.append(result)

if results:
    print("성공!!")
excel_column = 10
write_wb = Workbook()
write_ws = write_wb.create_sheet('result.xls')
for data in results:
    write_ws = write_wb.active
    write_ws.append(data)
write_wb.save('C:/Users/kjeng/강의평 크롤링/result.csv')



# Naver 페이 들어가기
# driver.get('https://order.pay.naver.com/home')
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')
# notices = soup.select('div.goods_item > div > a > p')
#
# for n in notices:
#     print(n.text.strip())
